# Sub Class 모델링

모델 이란 것은 Input을 Output으로 만들어주는 수식이다. 

해당 기능을 수행하는 두 가지 클래스가 `tf.keras.layers.Layer` 와 `tf.keras.layers.Model` 클래스이다. 

두가지 모두 **연산을 추상화** 하는 것으로 동일한 역할을 하지만, `tf.keras.layers.Model` 클래스의 경우 모델을 저장 하는 기능 과 `fit`, `save` 함수를 사용할 수 있다는 점에서 차이가 있다. 

- tf.keras.layers.Layer
- tf.keras.layers.Model

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
np.random.seed(7777)
tf.random.set_seed(7777)

### Linear Regression을 Layer로 만들어 보자. 

In [10]:
class LinearRegression(tf.keras.layers.Layer):
    def __init__(self, units):      # units 은 몇개의 output을 나타내는지를 나타냄
        super(LinearRegression, self).__init__()
        self.units = units

    def build(self, input_shape):       
        self.w = self.add_weight(       # 레이어 클래스에 들어있는 하이 메서드
            shape = (input_shape[-1], self.units),   # feature 개수를 받아옴.
            initializer = 'random_normal',
            trainable = True
        )  

        self.b = tf.Variable(0.0)


        # self.w = tf.Variable()
        # y_hat = w * input * b   # y_hat : y 추정치
    def call(self, inputs):  # overiding이라고도 한다.
        return tf.matmul(inputs, self.w) + self.b

### 가상 데이터

In [11]:
W_true = np.array([3., 2., 4., 1.]).reshape(4, 1)
B_true = np.array([1.])         # 정답

In [12]:
X = tf.random.normal((500, 4))
noise = tf.random.normal((500, 1))

y = X @ W_true + B_true + noise

In [13]:
opt = tf.keras.optimizers.SGD(learning_rate=0.03)

linear_layer = LinearRegression(1)

In [14]:
for epoch in range(100):
    with tf.GradientTape() as tape:
        y_hat = linear_layer(X)
        loss = tf.reduce_mean(tf.square(y - y_hat))

    grads = tape.gradient(loss, linear_layer.trainable_weights)

    # w.assign_sub(lr*dw)
    opt.apply_gradients(zip(grads, linear_layer.trainable_weights))
    
    if epoch % 10 == 0:
        print('epoch : {} loss : {}'.format(epoch, loss.numpy()))

epoch : 0 loss : 30.994300842285156
epoch : 10 loss : 10.11658000946045
epoch : 20 loss : 3.7557129859924316
epoch : 30 loss : 1.8134708404541016
epoch : 40 loss : 1.2191704511642456
epoch : 50 loss : 1.0369535684585571
epoch : 60 loss : 0.9809763431549072
epoch : 70 loss : 0.9637481570243835
epoch : 80 loss : 0.9584361910820007
epoch : 90 loss : 0.9567957520484924


In [8]:
linear_layer.trainable_weights

[<tf.Variable 'linear_regression/Variable:0' shape=(4, 1) dtype=float32, numpy=
 array([[ 0.08569323],
        [ 0.00562383],
        [-0.02817372],
        [-0.05486977]], dtype=float32)>,
 <tf.Variable 'linear_regression/Variable:0' shape=() dtype=float32, numpy=0.0>]

### ResNet - Sub Class 로 구현 하기 

1. Residual Block - Layer
2. ResNet  - Model

In [15]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

In [16]:
class ResidualBlock(tf.keras.layers.Layer):

    def __init__(self, filters=32, filter_match=False):
                        # residualblock 에 필요한 filters
                                    # filter_match가 True이면 

        super(ResidualBlock, self).__init__()

        self.conv1 = Conv2D(filters, kernel_size=1, padding='same', activation = 'relu')
        self.conv2 = Conv2D(filters, kernel_size=3, padding='same', activation = 'relu')
        self.conv3 = Conv2D(filters, kernel_size=1, padding='same', activation = 'relu')
        self.add = Add()
        self.filters = filters
        self.filter_match = filter_match
        
        if self.filter_match:    # True일때
            self.conv_ext = Conv2D(filters, kernel_size=1, padding='same')

    def call(self, inputs):
        net1 = self.conv1(inputs)
        net2 = self.conv2(net1)
        net3 = self.conv3(net2)
        
        if self.filter_match:
            res = self.add([self.conv_ext(inputs), net3])

        else:
            res = self.add([inputs, net3])
        
        return res

In [17]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()

        self.conv1 = Conv2D(32, kernel_size=3, strides=2, padding='same', activation = 'relu')
        self.maxp1 = MaxPool2D()
        self.block1 = ResidualBlock(64, True)
        self.block2 = ResidualBlock(64)
        self.maxp2 = MaxPool2D
        self.flat = Flatten()
        self.dense = Dense(num_classes)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.maxp1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.maxp2(x)
        x = self.flat(x)
        return self.dense(x)
        
        

In [18]:
model = ResNet(10)

### 학습 시켜보기